In [26]:
import pandas as pd
from tqdm.auto import tqdm

In [2]:
df = pd.read_csv('data/pair_data.csv')

In [3]:
topic_df = pd.read_csv('data/topics.csv')
content_df = pd.read_csv('data/content.csv')

In [4]:
content_id_title_map = content_df.set_index('id')['title'].to_dict()
topic_id_title_map = topic_df.set_index('id')['title'].to_dict()

In [5]:
# number of topics
len(df)

61517

In [6]:
df['num_pos'] = df.content_ids.map(lambda x: len(x.split(' ')))
df['num_neg'] = df.negative_content_ids.map(lambda x: len(x.split(' ')) if not pd.isnull(x) else 0)

In [7]:
df['num_pos'].sum()

279919

In [8]:
df['num_neg'].sum()

1173241

In [10]:
df

,topic_id,content_ids,negative_content_ids,score_negatives,num_pos,num_neg
0,t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...,c_27805efa8f05 c_f176727c6899 c_f128e8137445 c...,2.0358248 1.8792031 1.9632967 2.2078516 1.9087...,4,20
1,t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...,c_a8b58676dd52 c_0e1ee4cc91c5 c_98ccf251c863 c...,2.224468 2.233823 2.2474635 2.306077 2.2626696...,4,20
2,t_00069b63a70a,c_11a1dc0bfb99,c_fac629ee0e31 c_8361130de2a0 c_f44647ff1797 c...,2.2904983 1.8679115 2.3206046 1.757813 2.24606...,1,20
3,t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...,c_d776ef1186c0 c_fcb4cb5c1639 c_8adc09a72dc5 c...,2.439308 2.5001316 2.4520617 2.4789743 2.45769...,5,20
4,t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4,c_4fa38f97cd23 c_34bdd9059500 c_94e13b30c098 c...,2.449342 2.5471208 2.5908763 2.859213 2.490837...,3,20
...,...,...,...,...,...,...
61512,t_fff830472691,c_61fb63326e5d c_8f224e321c87,c_27cb05ee7d70 c_a3ca1a16a03d c_d1861e837fde c...,2.0429325 2.032765 1.9975747 2.0870557 2.04101...,2,18
61513,t_fff9e5407d13,c_026db653a269 c_0fb048a6412c c_20de77522603 c...,c_e1ef19ee9a52 c_a8411d9a68c4 c_1aab6afc3fc8 c...,1.9265834 1.9903316 1.8327581 1.8609569 1.9316...,10,20
61514,t_fffbe1d5d43c,c_46f852a49c08 c_6659207b25d5,c_e69642e64f9b c_508b4cbfa41e c_f09b32ab8bd8 c...,2.378184 2.2451966 2.3500276 2.6943378 2.27775...,2,20
61515,t_fffe14f1be1e,c_cece166bad6a,c_54fc13b8be96 c_0aff2d6967d9 c_15c2182d7171 c...,3.1012251 3.1012251 3.1012251 3.1012251 3.1012...,1,20


In [29]:
selected_contents = []
seletected_topics = []
labels = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    content_ids = row['content_ids']
    topic_id = row['topic_id']
    content_id = content_ids.split(' ')[0]
    selected_contents.append(content_id)
    seletected_topics.append(topic_id)
    labels.append(1)

    if pd.notnull( row['negative_content_ids']):
        neg_content_ids = row['negative_content_ids'].split(' ')
        neg_scores = row['score_negatives'].split(' ')

        neg_df = pd.DataFrame({'id': neg_content_ids, 'score':neg_scores})
        neg_df.columns = ['id', 'score']
        neg_df = neg_df.sort_values('score', ascending=False)
        neg_content_id = neg_df['id'].iloc[0]
        selected_contents.append(neg_content_id)
        seletected_topics.append(topic_id)
        labels.append(0)
    # break

100%|██████████| 61517/61517 [00:24<00:00, 2467.79it/s]


In [30]:
df = pd.DataFrame({'topic_id':seletected_topics, 'content_id':selected_contents, 'match':labels})

In [43]:
df

,topic_id,content_id,match,fold
0,t_00004da3a1b2,c_1108dd0c7a5d,1,1.0
1,t_00004da3a1b2,c_774459be08d5,0,3.0
2,t_00068291e9a4,c_639ea2ef9c95,1,3.0
3,t_00068291e9a4,c_152c0b174193,0,0.0
4,t_00069b63a70a,c_11a1dc0bfb99,1,3.0
...,...,...,...,...
123024,t_fffbe1d5d43c,c_0509c1059da0,0,2.0
123025,t_fffe14f1be1e,c_cece166bad6a,1,1.0
123026,t_fffe14f1be1e,c_f92cdc70f3d8,0,0.0
123027,t_fffe811a6da9,c_92b8fad372ee,1,3.0


In [46]:
df['content_title'] = df['content_id'].map(content_id_title_map)
df['topic_title'] = df['topic_id'].map(topic_id_title_map)

In [48]:
df

,topic_id,content_id,match,fold,content_title,topic_title
0,t_00004da3a1b2,c_1108dd0c7a5d,1,1.0,Молив като резистор,Откриването на резисторите
1,t_00004da3a1b2,c_774459be08d5,0,3.0,A New Find,Откриването на резисторите
2,t_00068291e9a4,c_639ea2ef9c95,1,3.0,Dados e resultados de funções: gráficos,Entradas e saídas de uma função
3,t_00068291e9a4,c_152c0b174193,0,0.0,Operations on Functions,Entradas e saídas de uma função
4,t_00069b63a70a,c_11a1dc0bfb99,1,3.0,Flow Charts: Logical..: Written Transcript of ...,Transcripts
...,...,...,...,...,...,...
123024,t_fffbe1d5d43c,c_0509c1059da0,0,2.0,Inscribed shapes,Inscribed shapes problem solving
123025,t_fffe14f1be1e,c_cece166bad6a,1,1.0,Juego con las palabras,Lección 7
123026,t_fffe14f1be1e,c_f92cdc70f3d8,0,0.0,Lección 7,Lección 7
123027,t_fffe811a6da9,c_92b8fad372ee,1,3.0,التقييم,تحديد العلاقة بين الإحداثيّات القطبية والإحداث...


In [49]:
from sklearn.model_selection import StratifiedKFold

In [50]:
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=67)

fold = 0
for train_inds, val_inds in kfold.split(df, df['match']):
    df.loc[val_inds, 'fold'] = fold
    fold += 1

In [51]:
df.groupby('fold').match.value_counts()

fold  match
0.0   1        15380
      0        15378
1.0   1        15379
      0        15378
2.0   1        15379
      0        15378
3.0   1        15379
      0        15378
Name: match, dtype: int64

In [52]:
df.to_csv('data/fold_split.csv', index=False)